In [3]:
#from nba_api.stats.static import players
#
#
# players.find_players_by_full_name("Lebron James")
#
#players.find_player_by_id(2544)

In [1]:
#Module load
from nba_api.stats.endpoints import commonallplayers #per avere l'elenco dei giocatori (id)
from nba_api.stats.endpoints import playercareerstats #per avere i dati
from nba_api.stats.endpoints import playerawards #per sapere chi ha vinto MVP
from nba_api.stats.endpoints import teamyearbyyearstats #classifiche finali delle squadre
import pandas as pd
from tqdm import tqdm
import pickle

In [2]:
from_year = 2010

total_players = commonallplayers.CommonAllPlayers().get_data_frames()[0]
total_players['TO_YEAR'] = total_players['TO_YEAR'].astype('int')
total_players = total_players[total_players['TO_YEAR'] > from_year]

#Ho controllato che in total_players, il PERSON_ID sia univoco
total_players

,PERSON_ID,DISPLAY_LAST_COMMA_FIRST,DISPLAY_FIRST_LAST,ROSTERSTATUS,FROM_YEAR,TO_YEAR,PLAYERCODE,PLAYER_SLUG,TEAM_ID,TEAM_CITY,TEAM_NAME,TEAM_ABBREVIATION,TEAM_CODE,TEAM_SLUG,GAMES_PLAYED_FLAG,OTHERLEAGUE_EXPERIENCE_CH
9,203518,"Abrines, Alex",Alex Abrines,0,2016,2018,alex_abrines,alex_abrines,0,,,,,None,Y,00
10,1630173,"Achiuwa, Precious",Precious Achiuwa,1,2020,2021,precious_achiuwa,precious_achiuwa,1610612761,Toronto,Raptors,TOR,raptors,raptors,Y,00
15,203112,"Acy, Quincy",Quincy Acy,0,2012,2018,quincy_acy,quincy_acy,0,,,,,None,Y,01
19,1629121,"Adams, Jaylen",Jaylen Adams,0,2018,2020,jaylen_adams,jaylen_adams,0,,,,,None,Y,01
20,203919,"Adams, Jordan",Jordan Adams,0,2014,2015,jordan_adams,jordan_adams,0,,,,,None,Y,01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4714,203092,"Zeller, Tyler",Tyler Zeller,0,2012,2019,tyler_zeller,tyler_zeller,0,,,,,None,Y,00
4720,1627757,"Zimmerman, Stephen",Stephen Zimmerman,0,2016,2016,stephen_zimmerman,stephen_zimmerman,0,,,,,None,Y,01
4721,1627835,"Zipser, Paul",Paul Zipser,0,2016,2017,paul_zipser,paul_zipser,0,,,,,None,Y,01
4722,1627790,"Zizic, Ante",Ante Zizic,0,2017,2019,ante_zizic,ante_zizic,0,,,,,None,Y,01


In [40]:
# total_names = total_players[["PERSON_ID", "DISPLAY_FIRST_LAST"]]
# total_stats = pd.DataFrame()

In [41]:
# stats = playercareerstats.PlayerCareerStats(player_id=2544).get_data_frames()[0]
# stats = stats.drop(stats.columns[[2, 4, 7]], axis=1)
# awards = playerawards.PlayerAwards(player_id=2544).get_data_frames()[0]
# awards = awards.drop(awards.columns[[1, 2, 3, 5, 7, 8, 9, 10, 11, 12, 13]], axis=1)
# awards = awards.rename(columns={"SEASON": "SEASON_ID"})
# awards = awards.rename(columns={"PERSON_ID": "PLAYER_ID"})
# awards = awards.loc[awards["DESCRIPTION"]=="NBA Most Valuable Player"]

In [34]:
all_teams_history = list(total_players["TEAM_ID"].drop_duplicates())
data_all_team_stats = pd.DataFrame()

for team in all_teams_history:
    t = teamyearbyyearstats.TeamYearByYearStats(team_id=team).get_data_frames()[0]
    data_all_team_stats = pd.concat([data_all_team_stats, t], axis=0, ignore_index=True)

#data_all_team_stats = [int("20" + x.split("-")[1]) for x in all_teams_history] NON SERVE, FACCIO DOPO -- ELIMINARE

data_all_team_stats = data_all_team_stats.rename(columns={"YEAR": "SEASON_ID"})
data_all_team_stats = data_all_team_stats[["TEAM_ID", "SEASON_ID", "CONF_RANK"]]

with open("data_all_team_stats.pkl",'wb') as f:
    pickle.dump(data_all_team_stats,f)

data_all_team_stats


,TEAM_ID,TEAM_CITY,TEAM_NAME,YEAR,GP,WINS,LOSSES,WIN_PCT,CONF_RANK,DIV_RANK,...,OREB,DREB,REB,AST,PF,STL,TOV,BLK,PTS,PTS_RANK
0,1610612761,Toronto,Raptors,1995-96,82,21,61,0.256,14,8,...,1071,2213,3284,1927,1987,745,1544,493,7994,21
1,1610612761,Toronto,Raptors,1996-97,82,30,52,0.366,12,8,...,1135,2254,3389,1714,1883,722,1347,517,7829,18
2,1610612761,Toronto,Raptors,1997-98,82,16,66,0.195,15,8,...,1187,2149,3336,1746,1851,769,1371,663,7781,17
3,1610612761,Toronto,Raptors,1998-99,50,23,27,0.460,10,6,...,712,1447,2159,1036,1139,439,799,321,4557,18
4,1610612761,Toronto,Raptors,1999-00,82,45,37,0.549,6,3,...,1098,2449,3547,1947,1989,666,1137,544,7968,17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1562,1610612742,Dallas,Mavericks,2017-18,82,24,58,0.293,13,4,...,666,2717,3383,1858,1578,578,1007,310,8390,28
1563,1610612742,Dallas,Mavericks,2018-19,82,33,49,0.402,14,5,...,832,2884,3716,1918,1650,533,1167,351,8927,21
1564,1610612742,Dallas,Mavericks,2019-20,75,43,32,0.573,7,2,...,791,2729,3520,1849,1462,460,953,363,8776,1
1565,1610612742,Dallas,Mavericks,2020-21,72,42,30,0.583,5,1,...,657,2463,3120,1647,1396,450,869,311,8096,17


In [ ]:
total_names = total_players[["PERSON_ID", "DISPLAY_FIRST_LAST"]]
total_stats = pd.DataFrame()
with open("data_all_team_stats.pkl", "rb") as f:
    data_all_team_stats = pickle.load(f)
data_all_team_stats = data_all_team_stats.rename(columns={"YEAR": "SEASON_ID"}) #prima ricaricare cella sopra e eliminare perché ridondante
data_all_team_stats = data_all_team_stats[["TEAM_ID", "SEASON_ID", "CONF_RANK"]] #prima ricaricare cella sopra e eliminare perché ridondante

for index, row in tqdm(total_names.iterrows()):
    stats = playercareerstats.PlayerCareerStats(player_id=row["PERSON_ID"]).get_data_frames()[0]
    #stats = playercareerstats.PlayerCareerStats(player_id=2544).get_data_frames()[0]

    stats = stats.drop(stats.columns[[2, 4, 7]], axis=1)

    awards = playerawards.PlayerAwards(player_id=row["PERSON_ID"]).get_data_frames()[0]
    #awards = playerawards.PlayerAwards(player_id=2544).get_data_frames()[0]

    awards = awards.drop(awards.columns[[0, 1, 2, 3, 5, 7, 8, 9, 10, 11, 12, 13]], axis=1) #Tolgo anche PERSON_ID
    awards = awards.rename(columns={"SEASON": "SEASON_ID"})
    #awards = awards.rename(columns={"PERSON_ID": "PLAYER_ID"}) #tolto perché dava interferenza
    awards = awards.loc[awards["DESCRIPTION"]=="NBA Most Valuable Player"]

    #merge
    stats=stats.merge(awards, how="left", on="SEASON_ID")

    #1 to MVP, 0 to other
    awards["DESCRIPTION"] = awards["DESCRIPTION"].replace("NBA Most Valuable Player", 1) #Replace the description MVP with 1
    awards["DESCRIPTION"] = awards["DESCRIPTION"].fillna(0) #Replace NULL with 0
    awards = awards.rename(columns={"DESCRIPTION": "MVP"})

    #merge della classifica in cui è arrivata la squadra in una certa stagione
    stats = stats.merge(data_all_team_stats, how="inner", on=["SEASON_ID", "TEAM_ID"])

    total_stats = pd.concat([total_stats, stats],axis=0,ignore_index=True)


#Tolgo la formattazione "year1-year2" (2010-11) e lascio solo il secondo anno (2011)
total_stats["SEASON_ID"] = [int("20"+x.split("-")[1]) for x in list(total_stats["SEASON_ID"])] #Problema se si prendono dati < anno 2000
#Prendo i dati superiore all'anno FROM_YEAR
total_stats= total_stats[total_stats["SEASON_ID"] > from_year]


with open("player_stats.pkl",'wb') as f:
    pickle.dump(total_stats,f)


20it [00:51,  1.91s/it]

In [24]:

#Find lebron james totals stats
#playercareerstats.PlayerCareerStats(player_id=2544).get_data_frames()[0]

total_stats

,PLAYER_ID,SEASON_ID,LEAGUE_ID,TEAM_ID,TEAM_ABBREVIATION,PLAYER_AGE,GP,GS,MIN,FGM,...,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PTS
0,203518,2016-17,00,1610612760,OKC,23.0,68,6,1055.0,134,...,0.898,18,68,86,40,37,8,33,114,406
1,203518,2017-18,00,1610612760,OKC,24.0,75,8,1134.0,115,...,0.848,26,88,114,28,38,8,25,124,353
2,203518,2018-19,00,1610612760,OKC,25.0,31,2,588.0,56,...,0.923,5,43,48,20,17,6,14,53,165
3,1630173,2020-21,00,1610612748,MIA,21.0,61,4,737.0,124,...,0.509,73,135,208,29,20,28,43,91,304
4,1630173,2021-22,00,1610612761,TOR,22.0,25,17,625.0,83,...,0.574,56,134,190,37,13,14,30,56,201
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9774,1627826,2018-19,00,1610612746,LAC,22.0,26,25,524.0,100,...,0.733,61,139,200,38,10,24,37,64,244
9775,1627826,2018-19,00,0,TOT,22.0,59,37,1039.0,212,...,0.802,115,247,362,63,14,51,70,137,525
9776,1627826,2019-20,00,1610612746,LAC,23.0,72,70,1326.0,236,...,0.747,197,346,543,82,16,66,61,168,596
9777,1627826,2020-21,00,1610612746,LAC,24.0,72,33,1609.0,257,...,0.789,189,330,519,90,24,62,81,187,650


#How it's structured this dataset
- Player_id --> id del giocatore(da ritrovare il nome con players.find_player_by_id())
- Season_id --> id della stagione
- ...other...
- GP --> Games Played
- GS --> Games ... (idk but similar to GP) (I think is not usefull)
- MIN Minutes Played
- FGM Field Goals Made --> Numero canestri fatti
- FGA Field Goals Attempted --> Numero tiri provati
- FG% Field Goal Percentage --> FGM/FGA, rapporto precisione tiri
- 3PM 3 Point Field Goals Made --> Numero canestri da 3 fatti
- 3PA 3 Point Field Goals Attempted --> Numero tiri da 3 provati
- 3P% 3 Point Field Goals Percentage --> 3PM/3PA, precisione tiri da 3
- FTM Free Throws Made --> Tiri liberi fatti
- FTA Free Throws Attempted --> tiri liberi provati
- FT% Free Throw Percentage --> FTM/FTA
- OREB Offensive Rebounds --> Rimbalzi conquistati in attacco
- DREB Defensive Rebounds --> rimbalzi conquistati in difesa
- REB Rebounds --> OREB + DREB, rimbalzi totali
- AST Assists
- TOV Turnovers --> palla persa
- STL Steals --> palla rubata
- BLK Blocks --> blocchi
- PF Personal Fouls --> falli personali
- DD2 Double Doubles --> doppie doppie
- TD3 Trible Doubles --> triple doppie
- PTS Points

In [23]:
with open("player_stats.pkl", "rb") as f:
    total_stats = pickle.load(f)

total_stats





0it [00:03, ?it/s]


ValueError: Can only compare identically-labeled Series objects

In [8]:
from nba_api.stats.endpoints import BoxScoreAdvancedV2

BoxScoreAdvancedV2.player_stats().get_data_frames()[0]

TypeError: 'NoneType' object is not callable

In [22]:
from nba_api.stats.endpoints import playerawards

playerawards.PlayerAwards(player_id=2544).get_data_frames()[0]

,PERSON_ID,FIRST_NAME,LAST_NAME,TEAM,DESCRIPTION,ALL_NBA_TEAM_NUMBER,SEASON,MONTH,WEEK,CONFERENCE,TYPE,SUBTYPE1,SUBTYPE2,SUBTYPE3
0,2544,LeBron,James,Cleveland Cavaliers,All-Defensive Team,1,2008-09,None,None,1610612739,Award,Kia Motors,KIADT,None
1,2544,LeBron,James,Cleveland Cavaliers,All-Defensive Team,1,2009-10,None,None,1610612739,Award,Kia Motors,KIADT,None
2,2544,LeBron,James,Miami Heat,All-Defensive Team,1,2010-11,None,None,1610612748,Award,Kia Motors,KIADT,None
3,2544,LeBron,James,Miami Heat,All-Defensive Team,1,2011-12,None,None,1610612748,Award,Kia Motors,KIADT,None
4,2544,LeBron,James,Miami Heat,All-Defensive Team,1,2012-13,None,None,1610612748,Award,Kia Motors,KIADT,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
144,2544,LeBron,James,Cleveland Cavaliers,NBA Sporting News Most Valuable Player of the ...,None,2005-06,None,None,None,Award,Sporting News,SNMVP,None
145,2544,LeBron,James,Cleveland Cavaliers,NBA Sporting News Rookie of the Year,None,2003-04,None,None,None,Award,Sporting News,SNROY,None
146,2544,LeBron,James,USA,Olympic Bronze Medal,None,2004,None,None,None,Award,Olympic,Bronze,None
147,2544,LeBron,James,USA,Olympic Gold Medal,None,2008,None,None,None,Award,Olympic,Gold,None
